In [2]:
#This file will contain the code I used to create my dataset (but excludes my personal API keys) and thus will not run
import csv
#first read the given data.csv file
file = open('xternData.csv')
csvreader = csv.reader(file)
header = []
header = next(csvreader)
rows = []
for row in csvreader:
        rows.append(row)
file.close()

In [ ]:
#Now let's just get the coWorking Locations
coWorkingLocations = []
for i in range(1,6):
    coWorkingLocations.append(rows[i][1])
coWorkingLocations

In [ ]:
#and the housing location
housingLocation = rows[0][1]
housingLocation

In [ ]:
#Now we will use the google distance API to get info about distance between locations
#my_api_key = ...
startAdd = housingLocation
url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial"
url2 = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?inputtype=textquery"
info = "types,business_status,formatted_address,geometry,name,place_id,photos"

distanceFromHousing = []
timeFromHousing = []
coWorkingLat = []
coWorkingLng = []
for i in coWorkingLocations:
    #print(i)
    endAdd = i
    results = requests.get(url + "&origins=" + startAdd + "&destinations=" + endAdd + "&key=" + my_api_key)
    distance = results.json()["rows"][0]["elements"][0]["distance"]["text"] 
    #print(distance)
    time = results.json()["rows"][0]["elements"][0]["duration"]["text"]
    #print(time)
    distanceFromHousing.append(distance)
    timeFromHousing.append(time)
    results2 = requests.get(url2 + "&input=" + endAdd + "&fields=" + info + "&key=" + my_api_key)
    #print(results.json().keys())
    coWorkingLat.append((results2.json()['candidates'][0]['geometry']['location']['lat']))
    coWorkingLng.append((results2.json()['candidates'][0]['geometry']['location']['lng']))


In [ ]:
#now let's get the resturants near each of the coworking locations 

payload={}
headers = {}
resturantDict = {}

for x in range(0,5):
    #print(x)
    lat = coWorkingLat[x]
    lng = coWorkingLng[x]
    url3 = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="+str(lat)+"%2C"+str(lng)+"&radius=1500&type=restaurant&key="+my_api_key
    results = requests.request("GET", url3, headers=headers, data=payload)
    for i in range(0,20):
        resturantName=(results.json()['results'][i]['name'])
        resturantLat=(results.json()['results'][i]['geometry']['location']['lat'])
        resturantLon=(results.json()['results'][i]['geometry']['location']['lng'])
        try: 
            resturantRating = (results.json()['results'][i]['rating'])
        except:
            resturantRating = 'none'
        try:
            resturantPriceLev = (results.json()['results'][i]['price_level'])
        except:
            resturantPriceLev = 'none'
        resturantDict[str(resturantName)] = [str(resturantLat),str(resturantLon),str(resturantRating),str(resturantPriceLev)]

#print(resturantDict)

In [ ]:
#Now let's write these results about resturants to a file so we can do analysis in R
import csv
csv_file = open('TechPointresturant.csv','w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Name','Lat','Lng','Rating','Price_Lev'])
#print(['Name','Lat','Lng','Rating','Price_Lev'])
for item in resturantDict:
    csv_writer.writerow([item,resturantDict[item][0],resturantDict[item][1],resturantDict[item][2],resturantDict[item][3]])
    #print ([item,resturantDict[item][0],resturantDict[item][1],resturantDict[item][2],resturantDict[item][3]])
csv_file.close()

In [ ]:
#First Let's Read in the Data.csv file
import csv
file = open('xternData.csv')
type(file)
csvreader = csv.reader(file)
header = []
header = next(csvreader)
header
data = []
for row in csvreader:
        data.append(row)
file.close()
#print(data)

In [ ]:
#Now Let's read in the Top Resturants!
file = open('topResturants.csv')
type(file)
csvreader = csv.reader(file)
header = []
header = next(csvreader)
header
top20 = []
for item in csvreader:
        top20.append(item)
file.close()
print(top20)

In [ ]:
#Now using the Google Distance API let's find the distance of each resturant from the housing location
#and from all of the coworking Locations 
#startAdd = input ("Enter a starting point:")
#endAdd =  input ("Enter the destination:")
import requests
import json
#my_api_key = ...
url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial"
finalTable={}

for item in top20:
    distanceArray=[]
    for i in range(6):
        startAdd = data[i][1]
        endAdd = str(item[1])+"%2C"+str(item[2])
        results = requests.get(url + "&origins=" + startAdd + "&destinations=" + endAdd + "&key=" + my_api_key)
        distance = results.json()["rows"][0]["elements"][0]["distance"]["text"] 
        distanceArray.append(distance)
    finalTable[item[0]]=distanceArray
print(finalTable)

In [ ]:
#Let's create a dictionary containg the events that are in the 10 Week Sample Plan
eventDict = {'Indy 500':'4790 W. 16th St., Indianapolis, IN 46222','Talbot Street Art Fair':'16th and Talbott Sts. Indianapolis, IN 46202','Indianapolis Indians Baseball Game':'501 W Maryland St, Indianapolis, IN 46225','5 Seconds of Summer Concert':'801 W. Washington St. Indianapolis, IN 46204','Indianapolis CyberSecurity Confrence':'50 W Fall Creek Pkwy N Dr, Indianapolis, IN 46208','Gen Con':'100 S. Capitol Ave., Indianapolis, IN 46225'}
#print(eventDict)

In [ ]:
#Let's create a table for events that includes the event name and distance from housing and coworking locations
url = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial"
eventTable={}

for item in eventDict:
    distanceArray=[]
    for i in range(6):
        startAdd = data[i][1]
        endAdd = eventDict[item]
        results = requests.get(url + "&origins=" + startAdd + "&destinations=" + endAdd + "&key=" + my_api_key)
        distance = results.json()["rows"][0]["elements"][0]["distance"]["text"] 
        #time = results.json()["rows"][0]["elements"][0]["duration"]["text"]
        distanceArray.append(distance)
    eventTable[item]=distanceArray
print(eventTable)

In [ ]:
#Let's get the distance location for housing too
housing=[]
for i in range(6):
    startAdd = data[i][1]
    endAdd = data[0][1]
    results = requests.get(url + "&origins=" + startAdd + "&destinations=" + endAdd + "&key=" + my_api_key)
    distance = results.json()["rows"][0]["elements"][0]["distance"]["text"] 
    housing.append(distance)
print(housing)

In [ ]:
#Let's write everything into a table now!
import csv
csv_file = open('theLocationData.csv','w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Location Name','HousingDist','CoWork1Dist','CoWork2Dist','CoWork3Dist','CoWork4Dist','CoWork5Dist'])
#print(['Name','Lat','Lng','Rating','Price_Lev'])
for item in eventTable:
    csv_writer.writerow([item,eventTable[item][0],eventTable[item][1],eventTable[item][2],eventTable[item][3],eventTable[item][4],eventTable[item][5]])
for item in finalTable:
    csv_writer.writerow([item,finalTable[item][0],finalTable[item][1],finalTable[item][2],finalTable[item][3],finalTable[item][4],finalTable[item][5]])
csv_writer.writerow(['IUPUI',housing[0],housing[1],housing[2],housing[3],housing[4],housing[5]])
csv_file.close()